In [1]:
!pip install tabula-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00


In [10]:
import tabula
import pandas as pd
from tabula.io import read_pdf
import os
import re
import warnings

In [12]:
# Creating a dictionary with keys as bank names and assigning indexes to each column
Bank_statement_features = {
   'ICICI':
   {
       'Date':0,
       'Particulars':1,
       'Chq.No.':2,
       'Withdrawals':3,
       'Deposits':4,
       'Auto Sweep':5,
       'Reverse Sweep':6,
       'Balance(INR)':7,
   },#can further add features according to "bank name as keys" of the dictionary
}

In [11]:
# Using Tabula(an open source library) to extract tables from a pdf with much accuracy
# here using tabula-py(Python wrapper of tabula)
#setting the values of various parameters
filepath = r"/content/ICICI_Bank_Statement.pdf"
rows_list = tabula.read_pdf(filepath,pages='1',silent=True,stream=True,lattice=True,guess=False,
                            area = (320.0,9.0,743.81,601.11),encoding='utf-8')
                            #area refers to the Portion of the page to analyze(top,left,bottom,right).
                            # calulated using tabula script exporter

                            #pandas_options={'header': None,'error_bad_lines': False,'warn_bad_lines': False}
                            #pandas_options not needed when multiple_tabels=True


#converting the lists together into a dataFrame
rows_df = pd.DataFrame(rows_list[0],
                       columns = ['Date','Particulars','Chq.No.','Withdrawals','Deposits',
                                  'Auto Sweep','Reverse Sweep','Balance(INR)','Credit','Debit'])

#Removing unnecessary columns and rows
rows_df.drop(['Auto Sweep','Reverse Sweep'],axis=1,inplace=True)
rows_df.drop([0],inplace=True)

# tackling those entries which contain delimeter like ','
for i in range(1,len(rows_df)+1):
    rows_df['Withdrawals'][i] = rows_df['Withdrawals'][i].replace(',','')
    rows_df['Deposits'][i] = rows_df['Deposits'][i].replace(',','')

#converting string values to float so that airthmetic analysis can be made
rows_df["Withdrawals"] = pd.to_numeric(rows_df["Withdrawals"], downcast="float",errors='ignore')
rows_df["Deposits"] = pd.to_numeric(rows_df["Deposits"], downcast="float",errors='ignore')
5
rows_df

<ipython-input-11-8927c3020f74>:25: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  rows_df['Withdrawals'][i] = rows_df['Withdrawals'][i].replace(',','')
<ipython-input-11-8927c3020f74>:25: SettingWithCopyWarning: 
A value is trying to be set 

,Date,Particulars,Chq.No.,Withdrawals,Deposits,Balance(INR),Credit,Debit
1,01-07-2019,BY CASH - BIJOLIA,NaN,0.0,2000.0,"18,674.45 Cr",NaN,NaN
2,02-07-2019,BIL/INFT/001743804629/NA/,NaN,0.0,3740.0,"22,414.45 Cr",NaN,NaN
3,02-07-2019,MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF...,NaN,0.0,3000.0,"25,414.45 Cr",NaN,NaN
4,03-07-2019,BY CASH-KACHHOLA,NaN,0.0,3000.0,"28,414.45 Cr",NaN,NaN
5,03-07-2019,MMT/IMPS/918418038560/d2h/HANSHRAJSO/BANK OF\r...,NaN,0.0,14000.0,"42,414.45 Cr",NaN,NaN
6,05-07-2019,CLG/COSMOS ENTERPRISES /BRK,5181.0,40000.0,0.0,"2,414.45 Cr",NaN,NaN
7,06-07-2019,MMT/IMPS/918710930334/d2h/HANSHRAJSO/BANK OF\r...,NaN,0.0,5000.0,"7,414.45 Cr",NaN,NaN
8,08-07-2019,MMT/IMPS/918814106619/d2h/HANSHRAJSO/BANK OF\r...,NaN,0.0,5000.0,"12,414.45 Cr",NaN,NaN
9,08-07-2019,BIL/INFT/001748219433/NA/,NaN,0.0,2150.0,"14,564.45 Cr",NaN,NaN
10,08-07-2019,UPI/918911468469/UPI/nandkishormali4/State Ban...,NaN,0.0,3000.0,"17,564.45 Cr",NaN,NaN


In [8]:
#same code when all the pages are need to be covered
aux_list = tabula.io.read_pdf(filepath,
                     guess=False, pages='all', stream=False ,silent=True ,encoding="utf-8",
                     area = ( 320.0,9.0,743.81,601.11 ))
                     #columns = (65.3,196.86,294.96,351.81,388.21,429.77))

aux_df = pd.DataFrame(aux_list[0],
                      columns = ['Date','Particulars','Chq.No.','Withdrawals','Deposits',
                                 'Auto Sweep','Reverse Sweep','Balance(INR)','Credit','Debit'])

aux_df.drop(['Auto Sweep','Reverse Sweep'],axis=1,inplace=True)
aux_df.drop([0],inplace=True)

for i in range(1,len(aux_df)):
    aux_df['Withdrawals'][i] = aux_df['Withdrawals'][i].replace(',','')
    aux_df['Deposits'][i] = aux_df['Deposits'][i].replace(',','')

aux_df["Withdrawals"] = pd.to_numeric(aux_df["Withdrawals"], downcast="float",errors='ignore')
aux_df["Deposits"] = pd.to_numeric(aux_df["Deposits"], downcast="float",errors='ignore')

aux_df

<ipython-input-8-062ee1769280>:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  aux_df['Withdrawals'][i] = aux_df['Withdrawals'][i].replace(',','')
<ipython-input-8-062ee1769280>:15: SettingWithCopyWarning: 
A value is trying to be set on a

,Date,Particulars,Chq.No.,Withdrawals,Deposits,Balance(INR),Credit,Debit
1,01-07-2019,BY CASH - BIJOLIA,NaN,0.0,2000.00,"18,674.45 Cr",NaN,NaN
2,02-07-2019,BIL/INFT/001743804629/NA/,NaN,0.0,3740.00,"22,414.45 Cr",NaN,NaN
3,02-07-2019,MMT/IMPS/918317526479/vidrc/MKELECTRON/BANK OF...,NaN,0.0,3000.00,"25,414.45 Cr",NaN,NaN
4,03-07-2019,BY CASH-KACHHOLA,NaN,0.0,3000.00,"28,414.45 Cr",NaN,NaN
5,03-07-2019,MMT/IMPS/918418038560/d2h/HANSHRAJSO/BANK OF\r...,NaN,0.0,14000.00,"42,414.45 Cr",NaN,NaN
6,05-07-2019,CLG/COSMOS ENTERPRISES /BRK,5181.0,40000.0,0.00,"2,414.45 Cr",NaN,NaN
7,06-07-2019,MMT/IMPS/918710930334/d2h/HANSHRAJSO/BANK OF\r...,NaN,0.0,5000.00,"7,414.45 Cr",NaN,NaN
8,08-07-2019,MMT/IMPS/918814106619/d2h/HANSHRAJSO/BANK OF\r...,NaN,0.0,5000.00,"12,414.45 Cr",NaN,NaN
9,08-07-2019,BIL/INFT/001748219433/NA/,NaN,0.0,2150.00,"14,564.45 Cr",NaN,NaN
10,08-07-2019,UPI/918911468469/UPI/nandkishormali4/State Ban...,NaN,0.0,3000.00,"17,564.45 Cr",NaN,NaN


In [9]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=aux_df)

https://docs.google.com/spreadsheets/d/1idhsQOEwVBTmFm7YdYrKoaDTh_O4yAQ5dwgwS10ykqM#gid=0
